In [0]:
import warnings 
warnings.filterwarnings('ignore')

In [0]:
from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Convolution2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers
from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import tensorflow

Using TensorFlow backend.


In [0]:
from google.colab  import drive
drive.mount('/content/gdrive', force_remount = True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
path = '/content/gdrive/My Drive/THI'

In [0]:
def get_optimizer():
  return Adam(lr = 0.0002, beta_1=0.5)

In [0]:
def get_generator(optimizer):
  generator = Sequential()
  generator.add(Dense(1152, input_dim = 100, kernel_initializer = initializers.RandomNormal(stddev=0.02)))
  generator.add(LeakyReLU(0.2))


  generator.add(Dense(2304))
  generator.add(LeakyReLU(0.2))

  
  generator.add(Dense(4608))
  generator.add(LeakyReLU(0.2))

 
  generator.add(Dense(2304, activation = 'tanh'))
  generator.compile(loss = 'binary_crossentropy', optimizer = optimizer)
  return generator

In [0]:
def get_discriminator(optimizer):
  discriminator = Sequential()
  discriminator.add(Dense(4608, input_dim = 2304, kernel_initializer= initializers.RandomNormal(stddev=0.02)))
  discriminator.add(LeakyReLU(0.2))    
  discriminator.add(Dropout(0.3))
 
  discriminator.add(Dense(2304))    
  discriminator.add(LeakyReLU(0.2))    
  discriminator.add(Dropout(0.3))
 
  discriminator.add(Dense(1152))    
  discriminator.add(LeakyReLU(0.2))    
  discriminator.add(Dropout(0.3))

  discriminator.add(Dense(1, activation='sigmoid'))
  discriminator.compile(loss = 'binary_crossentropy', optimizer=optimizer)
  return discriminator

In [0]:
def get_GAN_network(discriminator, random_dim, generator, optimizer):
  discriminator.trainable = False 
  
  gan_input = Input(shape=(random_dim,))  

  x = generator(gan_input)      
  
  gan_output = discriminator(x)    
  
  gan = Model(inputs=gan_input, outputs=gan_output)    
  
  gan.compile(loss='binary_crossentropy', optimizer=optimizer)    
  
  return gan

In [0]:
def plot_generated_images(epoch, generator, examples = 100, dim = (10,10), figsize = (10,10)):
  noise = np.random.normal(0,1,size = [examples, 100])

  generated_images = generator.predict(noise)    
  generated_images = generated_images.reshape((examples, 48, 48))
  plt.figure(figsize=figsize)    
  for i in range(generated_images.shape[0]):        
    plt.subplot(dim[0], dim[1], i+1)        
    plt.imshow(generated_images[i], interpolation='nearest')        
    plt.axis('off')    
  plt.tight_layout()
  plt.savefig(path+'/GAN_1/gan_generated_image_epoch_%d.png' % epoch)


In [0]:
import pandas as pd
emotion = pd.read_csv(path + '/emotion.csv')
emotion.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [0]:
import numpy as np
def reshape_string_listofnumber(df, index):
  string = df[index]
  string = string.split(' ')
  lst = np.array([int(charac) for charac in string])

  return lst



In [0]:
def reshape_dataset_matrix(df):
  matrix = np.empty((df.shape[0], 48, 48, 1))
  for i in range(df.shape[0]):
    ele = reshape_string_listofnumber(df, i)
    ele = np.reshape(ele,(48,48,1))
    matrix[i] = ele

  return matrix

In [0]:
def train(epochs = 1, batch_size = 128):
  x_train = emotion.pixels[emotion.Usage == 'Training']
  x_test = emotion.pixels[emotion.Usage == 'PrivateTest'].reset_index(drop = True)

  x_train = reshape_dataset_matrix(x_train)
  x_test = reshape_dataset_matrix(x_test)

  x_train = (x_train.astype(np.float32) - 127.5)/127.5
  x_test = (x_test.astype(np.float32) - 127.5)/127.5

  x_train = np.reshape(x_train, (x_train.shape[0], 2304))
  x_test = np.reshape(x_test, (x_test.shape[0], 2304))

  batch_count = x_train.shape[0]/batch_size

  adam = get_optimizer()
  generator = get_generator(adam)
  discriminator = get_discriminator(adam)
  gan = get_GAN_network(discriminator, 100, generator, adam)

  for e in range(1, epochs+1):
    print('-'*10, 'Epoch %d' %e, '-'*10)
    for _ in tqdm(range(int(batch_count))):
      noise = np.random.normal(0, 1, size = [batch_size, 100])
      image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

      generated_images = generator.predict(noise)

      X = np.concatenate([image_batch, generated_images])
      y_dis = np.zeros(2*batch_size)
      y_dis[:batch_size] = 0.9

      discriminator.trainable = True
      discriminator.train_on_batch(X, y_dis)

      noise = np.random.normal(0, 1, size = [batch_size, 100])
      y_gen = np.ones(batch_size)
      discriminator.trainable = False
      gan.train_on_batch(noise, y_gen)

    if e == 1 or e%20 == 0:
      plot_generated_images(e, generator)

In [0]:
train(400,128)